In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [60]:
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import tensorflow as tf
import keras_tuner as kt

In [48]:


## Detect hardware (CPU/GPU/TPU), setup environment and return appropriate distribution strategy

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu='local') # set tpu is local as it should be available in the VM
    print('✅ Running on TPU ', tpu.master())
except:
    print('❌ Using CPU/GPU')
    tpu = None

if tpu:
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)



❌ Using CPU/GPU
REPLICAS:  1


2025-07-25 08:46:50.483957: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [37]:
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
oil_price = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')

In [38]:
#preprocessing holidays data
holidays.head()
#filtering out transferred holidays
holidays = holidays[holidays['transferred'] == False]
holidays = holidays.replace(to_replace=['Transfer', 'Bridge', 'Additional'], value='Holiday')
holidays['Holiday_Gen'] = holidays['type']=='Holiday' #holiday or not
holidays['Event_Gen'] = holidays['type']=='Event' #event or not
holidays['Work_Day_Gen'] = holidays['type']=='Work Day'
holidays.drop(columns=["type", "transferred"], inplace=True)

In [39]:
#preprocessing oil price data
oil_price.loc[0, "dcoilwtico"] = oil_price.loc[1, "dcoilwtico"] #fill in the only nan
oil_price.ffill(inplace=True)
oil_price.bfill(inplace=True)

In [40]:
#data preprocessing

scalers = {'scaler_onpromotion' : MinMaxScaler(), 'scaler_day' : MinMaxScaler(), 'scaler_month' : MinMaxScaler(),
           'scaler_year' : MinMaxScaler(), 'scaler_dcoilwtico' : MinMaxScaler(), 'scaler_DoW' : MinMaxScaler()}

def preprocess_data(data, stores, transactions, holidays, oil_price, fit_scaler, scalers):
    #data will be empty before adding all dfs
    #merge stores into data to substitute store ids with location data
    data = data.merge(stores, how = 'left', left_on = 'store_nbr', right_on = 'store_nbr')

    data = data.merge(holidays, how = 'left', left_on = 'date', right_on = 'date')
    data['Holiday'] = ((data['Holiday_Gen'] == True)
                      & ((data['city'] == data['locale_name'])
                        | (data['state'] == data['locale_name'])
                        | ('Ecuador' == data['locale_name']))).astype(dtype='float64')
    data['Event'] = ((data['Event_Gen'] == True)
                       & ((data['city'] == data['locale_name'])
                          | (data['state'] == data['locale_name'])
                          | ('Ecuador' == data['locale_name']))).astype(dtype='float64')
    data['Work_Day'] = ((data['Work_Day_Gen'] == True)
                        & ((data['city'] == data['locale_name'])
                           | (data['state'] == data['locale_name'])
                           | ('Ecuador' == data['locale_name']))).astype(dtype='float64')

    #add oil prices to data + fill missing values
    data = data.merge(oil_price, how = 'left', left_on = 'date', right_on = 'date')
    data.sort_values('id', inplace = True) #sort rows by id, modify original df with no copy made
    data['dcoilwtico'] = data['dcoilwtico'].ffill() #forward fill

    #add transactions to data
    data = data.merge(transactions, how = 'left', left_on = ['date', 'store_nbr'], right_on = ['date', 'store_nbr']) #join data and transactions  where date and store_nbr match

    #modify date, pay date info
    data['date'] = pd.to_datetime(data['date']) #turn into math friendly datetime data
    data['day'] = data['date'].dt.day #extract the day from the date and put in a column
    data['month'] = data['date'].dt.month #extract month and put in a column
    data['year'] = data['date'].dt.year #extract year and put in a column
    data['day_of_week'] = data['date'].dt.day_name() #get day of the week NAME (strings)
    data['DoW'] = data['date'].dt.dayofweek #get day of the week as number (monday = 0, int)
    #pay days are on the first and 16th
    data['Pay_Day'] = ((data['date'].dt.day == 1) | (data['date'].dt.day == 16)).astype(dtype = 'float64') #check if the day is 1 or 16 (if its payday)
    #there was an earthquake on 2016-04-16
    data['Earthquake'] = (data['date'] == '2016-04-16').astype(dtype = 'float64')

    #fill nans with proper values
    data.fillna({
        'Holiday_Gen': False, #fill all holiday gen nans with false etc
        'Event_Gen': False,
        'Work_Day_Gen': False,
        'Holiday': 0.0,
        'Event': 0.0,
        'Work_Day': 0.0,
        'Pay_Day': 0.0,
        'Earthquake': 0.0,
        'transactions': 0.0
    }, inplace=True) #change original df directly, no copy is made

    #onehotencoding categorical features: 
    encoder = OneHotEncoder()
    One_Hot_Encoding = pd.DataFrame(encoder.fit_transform(data[['store_nbr', 'family', 'day_of_week']]).toarray(),
                                   columns = encoder.get_feature_names_out())
    #creates a dataframe of the array output(toarray) of the encoder results

    #scaling numerical values

    #if a specific FIT scaler is given, fitting it then transforming it outside the if
    if fit_scaler: 
        scalers['scaler_dcoilwtico'].fit(oil_price[['dcoilwtico']]) #MinMaxScaler fit on dcoilwtico from oil_price
        for col in ['onpromotion', 'day', 'month', 'year', 'DoW']:
            scalers[f'scaler_{col}'].fit(data[[col]])
    data[['dcoilwtico']] = scalers['scaler_dcoilwtico'].transform(data[['dcoilwtico']])
    for col in ['onpromotion', 'day', 'month', 'year', 'DoW']:
        data[[col]] = scalers[f'scaler_{col}'].transform(data[[col]])

    #feature vectors and labels vector
    features = data[['onpromotion', 'dcoilwtico', 'Holiday', 'Event', 'Work_Day', 'Earthquake', 'day', 'month', 'year']]
    features = features.merge(One_Hot_Encoding, how = 'left', left_index = True, right_index = True) #add encoded features to features
    #left_index = the left joining key will be the index
    #right_index = the right joining key will be the index

    if 'sales' in data.columns:
        labels = data[['sales']]
    else: labels = []


    return features, labels, scalers, data


In [41]:

trainvaltes_feat, trainvaltes_label, scalers, datat = preprocess_data(train, stores, transactions, holidays, oil_price, True, scalers)
#true for fit scalers

test_feat, test_label, _, data = preprocess_data(test, stores, transactions, holidays, oil_price, False, scalers)
#false for fit scalers because they have already been fitted for the train data

/tmp/ipykernel_36/1817755179.py:46: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna({
/tmp/ipykernel_36/1817755179.py:46: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna({


In [42]:
# split into train val tes
train_feat, val_feat, train_label, val_label = train_test_split(trainvaltes_feat, trainvaltes_label, test_size = 0.005, random_state = 33)
#test_size = validation size

In [44]:
#models
svr = SVR()
rfr = RandomForestRegressor(n_estimators = 100)
kneigh = KNeighborsRegressor(n_neighbors = 5)
gbr = GradientBoostingRegressor(n_estimators = 200)
xbg = xgb.XGBRegressor()

#model space
EstimatorStr = {1 : 'svr', 2 : 'rfr', 3 : 'kneigh', 4 : 'gbr', 5 : 'xgb', 6 : 'tbd'}
EstimatorMdl = {1 : svr, 2 : rfr, 3 : kneigh, 4 : gbr, 5 : xgb}

#hyperparameter space
#grid spaces = include multiple combinations of values
#single spaces = staatic combinations

param_grid_svr = {'svr__C' : np.linspace(1, 10, 5), #C: Regularization strength. higher = less regularization
                  'svr_degree' : np.arange(2, 6, 1), #degree is only used for poly kernel. including for flexibility
                  'svr_kernel' : ['rbf']} #kernel (radial basis function)
param_single_svr = {'svr__C': [1],
                   'svr_degree' : [3],
                   'svr_kernel' : ['rbf']}

param_grid_rfr = {'randomforestregressor__n_estimators' : np.arange(50, 250, 50)}
param_single_rfr = {'randomforestregressor__n_estimators' : [100]}

param_grid_neigh = {'kneighborsregressor__n_neihbors' : np.arange(2, 12, 2)}
param_single_neigh = {'kneighborsregressor__n_neihbors' : [5]}

param_grid_gbr = {'gradienboostingregressor__n_estimators' : np.arange(50, 250, 50),
                 'gradientboostingregressor__max_depth' : np.arange(2, 6, 1)}
param_single_gbr = {'gradientboostingregressor__n_estimators' : [100],
                   'gradientboostingregressor__max_depth' : [3]}

param_grid_xgb = {'xgbregressor__n_estimators' : np.arange(50, 250, 50),
                 'xgbregressor__max_depth' : np.arange(2, 6, 1)}
param_single_xgb = {'xgbregressor__n_estimators' : [100], 'xgbregressor__max_depth' : [3]}

In [56]:
#Neural network model (deep neural network)
def build_network(hp):
    #define the input layer and what the model expects
    inp = tf.keras.Input(shape = (len(train_feat.columns),),name = 'input')

    #apply a FC (fully connected) layer then a regression

    reg = None #no regularizer
    #regularizers help reduce overfitting, memorizing etc
    do = hp.Float(name = 'dropout', min_value = 0.04, max_value = 0.04, default = 0.04) #dropout rate 0.04 = 4% train data thrown
    x = inp
    layers = hp.Int(name = 'layers', min_value = 5, max_value = 5, default = 5) #how many hidden layers
    units = hp.Int(name = 'units', min_value = 768, max_value = 768, default = 768) #how many neurons per layer
    act = 'relu' #Rectified Linear Unit, it keeps positive values, kills the negatives.

    for i, units in enumerate([units for i in range(layers)]):
        x = tf.keras.layers.Dense(units, activation = act, kernel_regularizer = reg, name=f'fc{i+1}')(x)
        x = tf.keras.layers.Dropout(do, name = f'do{i+1}')(x)
    out = tf.keras.layers.Dense(1, activation = act, kernel_regularizer = reg, name = 'regression')(x) #final regression output
    model = tf.keras.Model(inputs = inp, outputs = out, name = 'DNN_Network') #wrap it up in a keras object model

    lr_tune = hp.Float(name = 'learning_rate', min_value = 5e-5, max_value = 5e-3, sampling = 'log', default = 5e-4) #1e-3
    optimizer = tf.keras.optimizers.Adam(lr_tune) #adam optimizer with tunable learning rate
    model.compile(optimizer = optimizer, loss = tf.keras.losses.msle, metrics = ['msle']) #mean squared logarithmic error

    return model

with strategy.scope():
    model_nn = build_network(kt.HyperParameters())
#distributed training under strategy 

model_nn.summary() #show layer by layer summary

Model: "DNN_Network"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 103)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 768)            │        79,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ do1 (Dropout)                   │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 768)            │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ do2 (Dropout)                   │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc3 (Dense)                     │ (None, 768)            │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ do3 (Dropout)                   │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc4 (Dense)                     │ (None, 768)            │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ do4 (Dropout)                   │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc5 (Dense)                     │ (None, 768)            │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ do5 (Dropout)                   │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ regression (Dense)              │ (None, 1)              │           769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,443,009 (9.32 MB)

 Trainable params: 2,443,009 (9.32 MB)

 Non-trainable params: 0 (0.00 B)

In [58]:
## Training configurations

TUNING = True
TRAINING = False
ML = False
NN = True

In [61]:
## Hyperparameter search or single fitting for machine learning methods

for i_TypEstimator in [1, 2, 3, 4, 5]: # Choosing models to tune {1: svc, 2: rfc, 3: kneigh, 4: gbc, 5: xbg} 1, 2, 3, 4, 5
    pipeline = make_pipeline(EstimatorMdl[i_TypEstimator])
    if TUNING and ML:
        param_grid = globals()[f'param_grid_{EstimatorStr[i_TypEstimator]}']
        grid = GridSearchCV(pipeline, param_grid, scoring='neg_mean_squared_log_error', verbose=3, cv=skf5)
        grid.fit(trainvaltes_feat, np.array(trainvaltes_label).ravel())
        param_single = grid.best_params_
        for item in param_single:
            param_single[item] = [grid.best_params_[item]]
    if not TUNING and ML:
        param_single = globals()[f'param_single_{EstimatorStr[i_TypEstimator]}']

    if ML:
        grid = GridSearchCV(pipeline, param_single, scoring='neg_mean_squared_log_error', verbose=3)
        grid.fit(train_feat, np.array(train_label).ravel())
        print(grid.best_params_)
        model_ml = grid.best_estimator_
        print(f'Train score: {model_ml.score(train_feat, np.array(train_label).ravel())}')
        print(f'Validation score: {model_ml.score(val_feat, np.array(val_label).ravel())}')
        globals()[f'model_{EstimatorStr[i_TypEstimator]}'] = model_ml

In [62]:
## Training parameters for DNN network

epochs = 100

# Callback functions
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=5, verbose=1, monitor='val_msle', restore_best_weights=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, verbose=1, monitor='val_msle', restore_best_weights=True)
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-5 * 10**(epoch / 20)) # Find starting learning

tuner = kt.RandomSearch(
    hypermodel=build_network,
    objective='val_msle',
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="tuner",
    project_name="StoreSales",
    distribution_strategy = strategy,
)

# tuner = kt.Hyperband(
#     hypermodel=build_network,
#     objective='val_msle',
#     max_epochs=50,
#     factor=3,
#     hyperband_iterations=1,
#     directory="tuner",
#     project_name="StoreSales",
#     distribution_strategy = strategy,
# )

tuner.search_space_summary()

Search space summary
Default search space size: 4
dropout (Float)
{'default': 0.04, 'conditions': [], 'min_value': 0.04, 'max_value': 0.04, 'step': None, 'sampling': 'linear'}
layers (Int)
{'default': 5, 'conditions': [], 'min_value': 5, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
units (Int)
{'default': 768, 'conditions': [], 'min_value': 768, 'max_value': 768, 'step': 1, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0005, 'conditions': [], 'min_value': 5e-05, 'max_value': 0.005, 'step': None, 'sampling': 'log'}


In [ ]:
## Train model by Tuner or Fit

if TUNING and NN:
    tuner.search(train_feat, train_label, validation_data=(val_feat, val_label), batch_size=1024,
                 epochs=epochs, callbacks=[lr_scheduler, early_stopping_cb])
    best_models = tuner.get_best_models(num_models=2)
    model_nn = best_models[0]
    model_nn.summary()
    tuner.results_summary()

if TRAINING and NN:
    history = model_nn.fit(train_feat, train_label, validation_data=(val_feat, val_label), batch_size=1024,
                        epochs=epochs, callbacks=[lr_scheduler, early_stopping_cb])
    model_nn.evaluate(val_feat, val_label)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
0.04              |0.04              |dropout
5                 |5                 |layers
768               |768               |units
0.00014788        |0.00014788        |learning_rate

Epoch 1/100
2968/2968 ━━━━━━━━━━━━━━━━━━━━ 447s 149ms/step - loss: 1.9085 - msle: 1.9085 - val_loss: 0.4164 - val_msle: 0.4164 - learning_rate: 1.4788e-04
Epoch 2/100
2968/2968 ━━━━━━━━━━━━━━━━━━━━ 438s 147ms/step - loss: 0.3696 - msle: 0.3696 - val_loss: 0.3017 - val_msle: 0.3017 - learning_rate: 1.4788e-04
Epoch 3/100
2968/2968 ━━━━━━━━━━━━━━━━━━━━ 441s 148ms/step - loss: 0.2741 - msle: 0.2741 - val_loss: 0.2371 - val_msle: 0.2371 - learning_rate: 1.4788e-04
Epoch 4/100
2968/2968 ━━━━━━━━━━━━━━━━━━━━ 442s 149ms/step - loss: 0.2293 - msle: 0.2293 - val_loss: 0.2132 - val_msle: 0.2132 - learning_rate: 1.4788e-04
Epoch 5/100
2968/2968 ━━━━━━━━━━━━━━━━━━━━ 438s 147ms/step - loss: 0.1986 - msle: 0.1986 - val_loss: 0.1868 - va

In [ ]:
## Plot learning curves

if TRAINING and NN:
    history_fil = {key: history.history[key] for key in ['msle', 'val_msle']}
    history_fil2 = {key: history.history[key] for key in ['loss', 'val_loss']}
    history_fil3 = {key: history.history[key] for key in ['learning_rate']}
                                                      
    pd.DataFrame(history_fil).plot()
    plt.ylabel("MSLE")
    plt.xlabel("epochs")
    plt.axis([3, len(history_fil['val_msle']), 0, history_fil['val_msle'][3]+0.1*history_fil['val_msle'][3]])
    pd.DataFrame(history_fil2).plot()
    plt.ylabel("Loss")
    plt.xlabel("epochs")
    plt.axis([3, len(history_fil2['val_loss']), 0, history_fil2['val_loss'][3]+0.1*history_fil2['val_loss'][3]])
    pd.DataFrame(history_fil3).plot()
    plt.ylabel("Learning rate")
    plt.xlabel("epochs")

In [ ]:
## Plot sales for selected store and product family

store = 44
family = 'AUTOMOTIVE' # SEAFOOD AUTOMOTIVE
store_train = f'store_nbr_{store}'
family_train = f'family_{family}'
datat_fil = datat[((datat['store_nbr']==store) & (datat['family']==family))]
train_feat_fil = train_feat[((train_feat[store_train]==True) & (train_feat[family_train]==True))]
train_pred_fil = pd.DataFrame(model_nn.predict(train_feat_fil), columns=['prediction'], index=train_feat_fil.index)
datat_fil = datat_fil.merge(train_pred_fil, how='left', left_index=True, right_index=True)
datat_fil.set_index(['date'], inplace=True)

start = 31
plt.figure()
datat_fil[['sales', 'prediction']][start:31+start].plot(title='Sales prediction of store {} for {} products'.format(str(store), family))
plt.figure()
datat_fil[['sales', 'prediction']][start:90+start].plot(title='Sales prediction of store {} for {} products'.format(str(store), family))
plt.figure()
datat_fil[['sales', 'prediction']][start:356+start].plot(title='Sales prediction of store {} for {} products'.format(str(store), family))

In [ ]:
models=[]
if ML:
    models = models + ['svc', 'rfc', 'kneigh', 'gbc', 'xbg']
if NN:
    models = models + ['nn']

submission_all = pd.DataFrame(data[['id']])
for mod in models:
    model = globals()[f'model_{mod}']
    submission = pd.DataFrame(data[['id']])
    test_prediction = pd.DataFrame(model.predict(test_feat), columns=['sales'])
    submission = submission.merge(test_prediction, how='left', left_index=True, right_index=True)
    submission.to_csv(f'submission_{mod}.csv', index=False)
    submission_all[[mod]] = submission[['sales']]
#submission_all[['Transported']] = pd.DataFrame((submission_all[['svc', 'rfc', 'gbc', 'xbg', 'nn']].sum(axis=1)/5)>0.5)
#submission_all[['PassengerId', 'Transported']].to_csv('submission_avg.csv', index=False)

In [ ]:


# # Plot learning curves for definition of start leraning rate
# lrs = 1e-6 * (10 ** (np.arange(100) / 20)) # Define the learning rate array
# plt.figure(figsize=(10, 6)) # Set the figure size
# plt.grid(True) # Set the grid
# plt.semilogx(lrs, history.history["loss"]) # Plot the loss in log scale
# plt.tick_params('both', length=10, width=1, which='both') # Increase the tickmarks size
# plt.axis([1e-5, 1e-1, 0, 1]) # Set the plot boundaries

